In [2]:
import torch
print("CUDA Available:", torch.cuda.is_available())
print("Device Name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU only")


CUDA Available: True
Device Name: Tesla T4


In [3]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.optim import AdamW
from torch.nn import CrossEntropyLoss
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# --------------------------
# Load and clean dataset
# --------------------------
data_file = r"/content/train_snli.txt"

df = pd.read_csv(data_file, delimiter="\t", header=None, names=["s1", "s2", "y"])
df.dropna(inplace=True)   # drop missing rows
print("Class balance:\n", df["y"].value_counts())

# split dataset
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
train_df, val_df = train_test_split(train_df, test_size=0.1, random_state=42)
print(train_df.shape, val_df.shape, test_df.shape)

# --------------------------
# Model + Tokenizer
# --------------------------
tokenizer = AutoTokenizer.from_pretrained("HuggingFaceTB/SmolLM-135M")
model = AutoModelForSequenceClassification.from_pretrained(
    "HuggingFaceTB/SmolLM-135M", num_labels=2
)

# add pad token if missing
tokenizer.add_special_tokens({"pad_token": "[PAD]"})
model.resize_token_embeddings(len(tokenizer))
model.config.pad_token_id = tokenizer.pad_token_id


# --------------------------
# Dataset class
# --------------------------
class PairDataset(Dataset):
    def __init__(self, frame, tok, max_len=192):
        self.frame = frame
        self.tok = tok
        self.max_len = max_len

    def __len__(self):
        return len(self.frame)

    def __getitem__(self, idx):
        row = self.frame.iloc[idx]
        s1, s2 = str(row["s1"]), str(row["s2"])
        enc = self.tok(
            s1, s2,
            padding="max_length",
            truncation=True,
            max_length=self.max_len,
            return_tensors="pt"
        )
        return {
            "input_ids": enc["input_ids"].squeeze(0),
            "attention_mask": enc["attention_mask"].squeeze(0),
            "labels": torch.tensor(row["y"], dtype=torch.long)
        }


def collate_batch(batch):
    return {
        "input_ids": torch.stack([x["input_ids"] for x in batch]),
        "attention_mask": torch.stack([x["attention_mask"] for x in batch]),
        "labels": torch.stack([x["labels"] for x in batch])
    }


# --------------------------
# Dataloaders
# --------------------------
train_ds = PairDataset(train_df, tokenizer)
val_ds = PairDataset(val_df, tokenizer)
test_ds = PairDataset(test_df, tokenizer)

train_loader = DataLoader(train_ds, batch_size=16, shuffle=True, collate_fn=collate_batch)
val_loader = DataLoader(val_ds, batch_size=16, shuffle=False, collate_fn=collate_batch)
test_loader = DataLoader(test_ds, batch_size=16, shuffle=False, collate_fn=collate_batch)

# --------------------------
# Training loop
# --------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model = torch.nn.DataParallel(model)

opt = AdamW(model.parameters(), lr=2e-5)
loss_fn = CrossEntropyLoss()


def train_loop(model, loader, vloader, optimizer, criterion, n_epochs=3):
    for ep in range(n_epochs):
        model.train()
        total_loss = 0
        for batch in tqdm(loader, desc=f"Epoch {ep+1}"):
            ids, mask, labels = batch["input_ids"].to(device), batch["attention_mask"].to(device), batch["labels"].to(device)
            out = model(ids, attention_mask=mask)
            loss = criterion(out.logits, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        print(f"Epoch {ep+1}: Train Loss = {total_loss/len(loader):.4f}")
        acc = eval_loop(model, vloader)
        print(f"Validation Acc = {acc:.4f}")


def eval_loop(model, loader):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for batch in loader:
            ids, mask, labels = batch["input_ids"].to(device), batch["attention_mask"].to(device), batch["labels"].to(device)
            out = model(ids, attention_mask=mask)
            preds = torch.argmax(out.logits, dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    return correct / total





Class balance:
 y
0    183964
1    183405
Name: count, dtype: int64
(264505, 3) (29390, 3) (73474, 3)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/831 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/724 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/538M [00:00<?, ?B/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at HuggingFaceTB/SmolLM-135M and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


In [4]:
# --------------------------
# Train
# --------------------------
train_loop(model, train_loader, val_loader, opt, loss_fn, n_epochs=1)

Epoch 1: 100%|██████████| 16532/16532 [3:35:37<00:00,  1.28it/s]


Epoch 1: Train Loss = 0.1301
Validation Acc = 0.9620


In [5]:
import torch
torch.save(model.state_dict(), 'model_after_1_epoch.pth')

In [8]:
# --------------------------
# Train
# --------------------------
#train_loop(model, train_loader, val_loader, opt, loss_fn, n_epochs=1)

In [7]:
#import torch
#torch.save(model.state_dict(), 'model_after_2_epoch.pth')

In [9]:

# --------------------------
# Final test eval
# --------------------------
def test_eval(model, loader):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for batch in loader:
            ids, mask, labels = batch["input_ids"].to(device), batch["attention_mask"].to(device), batch["labels"].to(device)
            out = model(ids, attention_mask=mask)
            preds = torch.argmax(out.logits, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    print(classification_report(all_labels, all_preds))


test_eval(model, test_loader)

# --------------------------
# Save model + tokenizer
# --------------------------
save_path = r"/content/"
model_to_save = model.module if hasattr(model, "module") else model
model_to_save.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)
print("Model saved at", save_path)


              precision    recall  f1-score   support

           0       0.96      0.97      0.96     36586
           1       0.97      0.96      0.96     36888

    accuracy                           0.96     73474
   macro avg       0.96      0.96      0.96     73474
weighted avg       0.96      0.96      0.96     73474

Model saved at /content/


In [10]:
# --------------------------
# Save model + tokenizer
# --------------------------
save_path = r"/content/model"
model_to_save = model.module if hasattr(model, "module") else model
model_to_save.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)
print("Model saved at", save_path)


Model saved at /content/model
